In [33]:
from astropy.io import ascii
import numpy as np
import astropy.units as u
import pandas as pd
from astropy.time import Time
from astropy.visualization import hist

from minicyclop.io import read_latest

In [7]:
t = ascii.read("/home/tim/MMT/minicyclop/data/MiniCyclop/Data/Last_Seeing_Data.txt", delimiter='|', 
names=['UT', 'MST', 'MJD', 'flux', 'seeing', 'r0'])

In [8]:
t

UT,MST,MJD,flux,seeing,r0
str21,str20,float64,float64,float64,float64
6/20/2022 11:55:13 AM,6/20/2022 4:55:13 AM,2459750.9966885,2064.0,1.08,105.4


In [17]:
Time(t['MJD'][0], scale='utc', format='jd').isot

'2022-06-20T11:55:13.886'

In [29]:
d = read_latest("/home/tim/MMT/minicyclop/data/MiniCyclop/Data/Last_Seeing_Data.txt")

In [30]:
d['obstime'] - Time.now()

<TimeDelta object: scale='tai' format='jd' value=-0.38014002413865405>

In [31]:
t = ascii.read("/home/tim/MMT/minicyclop/data/MiniCyclop/Data/Seeing_Data.txt", delimiter='|', 
names=['UT', 'MST', 'MJD', 'flux', 'seeing', 'r0'])

In [40]:
df = t.to_pandas().set_index('UT').drop(columns=['MST', 'MJD'])

In [41]:
df

,flux,seeing,r0
UT,,,
6/8/2022 3:32:32 AM,2733.7,0.84,135.8
6/8/2022 3:33:27 AM,2755.7,0.84,134.4
6/8/2022 3:34:23 AM,2801.6,2.38,47.7
6/8/2022 3:35:16 AM,2853.2,1.01,112.3
6/8/2022 3:36:00 AM,2739.0,1.03,110.7
...,...,...,...
6/21/2022 10:02:42 AM,1448.8,1.04,109.3
6/21/2022 10:03:40 AM,1354.4,1.17,97.1
6/21/2022 10:05:51 AM,779.8,1.13,100.5
